In [1]:
# Importando os bibliotecas
import pandas as pd
from pandas import read_excel
import re
from concurrent.futures import ThreadPoolExecutor
from random import shuffle
import json
from bs4 import BeautifulSoup

### Carregando o dataset  

inicialmente vamos utilizar o corpus público do [Petrolês](https://petroles.puc-rio.ai/) sem as teses do IBICT, com o preprocessamento disponibilizado (apenas eliminação de ruídos, números e caracteres especiais). Ao final, a escolha do corpus deve ser reavaliada. 

In [ ]:
file = 'corpusPublico(sem IBICT)-SemProcessamento.txt'
text = open(file, "r", encoding='utf-8').read()
text = text.split('\n')
shuffle(text)

In [ ]:
print('Número de sentenças: ', len(text))

Número de sentenças:  4479566


Carregando lista de entidades nomeadas

In [ ]:
# Carregando arquivo
entidades = pd.read_csv('Entidades.csv', encoding='iso-8859-1', delimiter=';')
# Excluindo linhas duplicadas
entidades = entidades.drop_duplicates().reset_index()[['Entidade','Classe']]
entidades

,Entidade,Classe
0,ACRE,Bacia
1,AFOGADOS DA INGAZEIRA,Bacia
2,AGUA BONITA,Bacia
3,ALAGOAS,Bacia
4,ALMADA,Bacia
...,...,...
1281,Age 5,Unidade_Cronoestratigrafica
1282,Epoch 2,Unidade_Cronoestratigrafica
1283,Age 4,Unidade_Cronoestratigrafica
1284,Age 3,Unidade_Cronoestratigrafica


In [ ]:
sentença_entidade = {}

def verificar_entidades(sent):

    lista_entidade = []

    for ent in list(entidades['Entidade']): #Usando instâncias da 
        TEXT =  ent.lower()
        
        if re.search(r'\b' + TEXT + r'\b', sent.lower()):
            lista_entidade.append(ent)
      
    if lista_entidade != []:

        sentença_entidade[len(sentença_entidade)] = {'Sentença': sent, 'Entidades': lista_entidade}

    return 

In [ ]:
with ThreadPoolExecutor(100) as pool:
    pool.map(verificar_entidades, text[:10])

In [ ]:
sentença_entidade

{}

In [ ]:
with open('sentença_entidade.json', 'w') as f:
    json.dump(sentença_entidade, f)

# Taxonomia de O&G. (Wograine, 2021)

In [2]:
with open("Taxonomia_O&G.html", "r", encoding='utf-8') as t:
    soup = BeautifulSoup(t, 'html.parser')

FileNotFoundError: ignored

In [ ]:
pai = []

n = 0
for div in soup.find_all('h3'):
    n = n +1
    pai.append(div.text.strip(' (1234567890)'))
    
    #if n > 20:
    #    break
    
len(pai)

7008

In [ ]:
filho = []

n = 0
for span in soup.find_all('span'):
    n = n +1
    filho.append(span.text.strip(' (1234567890)'))
    
    #if n > 20:
    #    break
    
len(filho)

20786

In [ ]:
# Potenciais entidades mencionadas de O&G
EM = pai + filho
len(EM)

27794

In [ ]:
sentença_entidade = {}

def verificar_entidades_2(sent):
    lista_entidade = []
    
    for ent in EM: #Usando lista de potenciais Entidades Mencionadas
        try:
            TEXT =  ent.lower()
            if re.search(r'\b' + TEXT + r'\b', sent.lower()):
                lista_entidade.append(ent)
        except:
            pass
        
    if lista_entidade != []:
        sentença_entidade[len(sentença_entidade)] = {'Sentença': sent, 'Entidades': lista_entidade}

    return 

In [ ]:
with ThreadPoolExecutor(100) as pool:
    pool.map(verificar_entidades_2, text[:50])

In [ ]:
sentença_entidade

{0: {'Sentença': 'Os valores de dureza apresentaram-se de acordo com a literatura em torno dos <NUMBER> GPa Figura <NUMBER>  ',
  'Entidades': ['acordo', 'EM', 'dureza', 's.', 'torno', 'literatura', 'OS']},
 1: {'Sentença': '<NUMBER> são ilustrados por um programa que permite prever o ponto de operação para diferentes freqüências do acionamento do motor do conjunto de BCSS  ',
  'Entidades': ['ponto',
   'motor',
   'operação',
   'programa',
   'conjunto',
   'são',
   'BCSS',
   'ponto de operação',
   'BCSS']},
 2: {'Sentença': 'Os trabalhos tais como Clean Coal Technology Development in  ',
  'Entidades': ['clean', 'OS']},
 3: {'Sentença': 'A c o n c l u s ã o d e B a t e s <NUMBER> f o i a d e q u e a c o m b i n a ç ã o d a t e o r i a d o j a t o a x i a l e o c o n c e i t o d e l i m i t e v e r t i c a l r e q u e r q u e a c o r r e n t e h o m o p i c n a l d e s e n v o l v a - s e p a r c i a l m e n t e s u b - a é r e a com c a a d a s d e t o p o a p l a i n a d a s I 

In [ ]:
with open('sentença_entidade_v2.json', 'w') as f:
    json.dump(sentença_entidade, f)